ML Final Project

Spring 2017 - NYU CUSP

Notebook reads in datetime converted CitiBike Data by month (e.g. Jan_citi.csv) from 2016 and converts gets the counts and means per month

Written in Python 2.7 (Anaconda)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
from geopandas import GeoDataFrame
import os
import subprocess
from shapely.geometry import Point
from datetime import datetime
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score as rs
from sklearn import svm, datasets
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import IsolationForest
from pandas.tseries.holiday import USFederalHolidayCalendar
from mpl_toolkits.mplot3d import Axes3D

from itertools import compress

np.random.seed(222)

%matplotlib inline
%pylab inline
pd.options.display.max_columns = 120
pd.options.display.max_rows = 120

Populating the interactive namespace from numpy and matplotlib


/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
`%matplotlib` prevents importing * from pylab and numpy


In [2]:
proj_folder = os.getcwd()

Check for spurious entires

## Remove spurious trip length rationale

From website: A $101 security deposit hold will be placed on your card when you purchase a pass. If you incur usage

fees, your card will be charged. The fee for a lost or stolen bike is $1200 (+ tax).


## Total Loss: 

Annual pass = 163/year + 101 Security fee + ($ 2.50 * min) = 5616 minutes
 
3 day Pass = 24 + 101 Security fee + ($ 2.50 * 15min) = 4031.25

Day Pass = 12 + 101 Security fee + ($4 * 15min) = 4076.25

Ride = 4 + 101 Security fee + ($4 * 15min) = 4106.25 

So remove anything longer than 5616 minutes, the maximum time before Total Loss and keep trip long than 1 minute

In [3]:

def citi_agg_stat_day(label):
    '''This function is supposed to: Further clean citi bike data, Group by day, 
    get daily count and mean and then export the files to prepare for a large merge'''
    #Import the csv
    getfile = "data/" + label + ".csv"
    citi_agg = pd.read_csv(getfile)
    #Drop column
    citi_agg.drop('Unnamed: 0', axis=1, inplace=True)
    #change seconds to minutes
    citi_agg['tripduration'] = citi_agg['tripduration'].apply(lambda x: x / 60)
    #Remove spurious trips
    citi_agg = citi_agg[(citi_agg.tripduration > 1) & (citi_agg.tripduration < 5616)]
    #DatTime again??
    citi_agg['starttime'] = pd.to_datetime(citi_agg['starttime'])
    citi_agg['stoptime'] = pd.to_datetime(citi_agg['stoptime'])
    #Count Trips per day 
    citi_agg['bike_date'] = citi_agg['starttime'].dt.date
    citi_day_count = citi_agg.groupby(['bike_date'], as_index=False)['tripduration'].count()
    citi_day_count.columns = ['bike_date', 'bike_trip_count']
    
    #take the mean of the taxi stats each day
    citi_stat_agg = citi_agg.groupby(['bike_date'], as_index=False).mean()
    
    #Merge the seperate DataFrames 
    citi_df_by_day = citi_stat_agg.merge(citi_day_count, on='bike_date')
    
    #Get the weekday
    citi_df_by_day['weekday'] = 0
    for i in range(len(citi_df_by_day)):
        citi_df_by_day['weekday'].iloc[i] = citi_df_by_day.bike_date.iloc[i].weekday()
        
    #Export to csv
    output = "data/" + label + "_df_by_day.csv"
    citi_df_by_day.to_csv(output)
    #citi_df_byday_all
    return citi_df_by_day 

In [4]:
def citi_daystation_count(label):
    '''This function is supposed to: Further clean citi bike data, do some grouping
    and calculations and then export the files to prepare for a large merge'''
    #Import the csv
    getfile1 = "data/" + label + ".csv"
    citi_agg1 = pd.read_csv(getfile1)
    #Drop column
    citi_agg1.drop('Unnamed: 0', axis=1, inplace=True)
    #change seconds to minutes
    citi_agg1['tripduration'] = citi_agg1['tripduration'].apply(lambda x: x / 60)
    #Remove spurious trips
    citi_agg1 = citi_agg1[(citi_agg1.tripduration > 1) & (citi_agg1.tripduration < 5616)]
    #DatTime again??
    citi_agg1['starttime'] = pd.to_datetime(citi_agg1['starttime'])
    citi_agg1['stoptime'] = pd.to_datetime(citi_agg1['stoptime'])
    #citi_agg['day_number'] = label.starttime.apply(lambda x: x.hour)
    citi_agg1['bike_date'] = citi_agg1['starttime'].dt.date
    
    
    #Count Trips per day 
    citi_agg1['bike_date'] = citi_agg1['starttime'].dt.date
    citi_day_count1 = citi_agg1.groupby(['bike_date', "start_station id"], as_index=False)['tripduration'].count()
    citi_day_count1.columns = ['bike_date', "start_station id",'bike_trip_count']
    return citi_day_count1 

## Aggregate by day and Export

In [5]:
columns =['bike_date', 'tripduration', 'start_station id', 'start_station_latitude', 'start_station_longitude', \
          'end_station_id', 'end_station_latitude','end_station_longitude', 'bikeid', 'birth_year', \
          'gender', 'start_hour', 'end_hour','bike_trip_count', 'weekday']

In [8]:
months = ["Jan_citi", "Feb_citi", "Mar_citi", "Apr_citi", "May_citi", "Jun_citi", "Jul_citi", "Aug_citi", \
          "Sep_citi", "Oct_citi", "Nov_citi", "Dec_citi"]

In [6]:
months2 = ["citibike_201501", "citibike_201502", "citibike_201503", "citibike_201504", "citibike_201505", \
          "citibike_201506", "citibike_201507", "citibike_201508", "citibike_201509", "citibike_201510", \
          "citibike_201511", "citibike_201512"]

In [7]:
#Create Empty dataframe
citi_df_all_day = pd.read_csv(io.StringIO(None), names=columns, dtype=dict(zip(columns,[object, float, float, float, float, float, float, float, float, float, float, float, float, int, int])))

In [10]:
# Create the files in bulk
# By day with count and mean
for i in (months2):
    x = citi_agg_stat_day(str(i))
    citi_df_all_day = citi_df_all_day.append(x, ignore_index = True)
        

/Users/christopherstreich/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
citi_df_all_day.to_csv('data/citi_df_all_day.csv')

In [80]:
citi_df_all_day.head()

,bike_date,tripduration,start_station id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bikeid,birth_year,gender,start_hour,end_hour,bike_trip_count,weekday


## Grouping by station and time of day

In [69]:
months = ["Jan_citi", "Feb_citi", "Mar_citi", "Apr_citi", "May_citi", "Jun_citi", "Jul_citi", "Aug_citi", \
          "Sep_citi", "Oct_citi", "Nov_citi", "Dec_citi"]

In [70]:
columns2 =['bike_date', 'start_station id', 'bike_trip_count', 'birth_year', \
          'gender', 'start_hour', 'end_hour', 'weekday']

In [71]:
citi_day_by_station = pd.read_csv(io.StringIO(None), names=columns2, dtype=dict(zip(columns,[object, int, int])))

In [72]:
#Get Station count grouped by day
for i in (months):
    y = citi_daystation_count(str(i))
    citi_day_by_station = citi_day_by_station.append(y, ignore_index = True)

In [77]:
citi_day_by_station.to_csv('data/citi_day_by_station.csv')

### Add holiday / day of week / weekend features

In [ ]:
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2016', return_name=True)
holidays.name = 'name'
holidays.head()

In [ ]:
holidays_all = pd.concat([holidays,
                         "Day Before " + holidays.shift(-1, 'D'),
                         "Day After " + holidays.shift(1, 'D')])
holidays_all = holidays_all.sort_index()

In [ ]:
#add holiday dummy variable
citi_day_by_station['holiday'] = 0
citi_day_by_station['holiday'][citi_day_by_station['bike_date'].isin(holidays_all.index.date)] = 1

In [ ]:
citi_day_by_station['day_of_week'] = citi_day_by_station.taxi_date.apply(lambda x: x.weekday())

In [ ]:
#control for Friday / Saturday nights
citi_day_by_station['weekend'] = 0
citi_day_by_station['weekend'][(citi_day_by_station['day_of_week'] == 4) | (citi_day_by_station['day_of_week'] == 5)] = 1

## Clustering Anomaly Detection

### K-means Anomaly Detection

In [ ]:
cluster_features = ['bike_trip_count', 'holiday', 'weekend']

In [ ]:
X = citi_day_by_station[cluster_features]
X.head(2)

In [ ]:
#normalize columns
for i in X.columns:
    X[i] = (X[i] - X[i].mean()) / X[i].std()
    
X.head()

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for n_clusters in range_n_clusters:
    km = KMeans(n_clusters=n_clusters, random_state=324)
    cluster_labels = km.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters ={},".format(n_clusters)+" the average silhouette_score is :{}".format(silhouette_avg))

In [ ]:
n=3
KM=KMeans(n_clusters=n,random_state=999)
scor=KM.fit_predict(X)

res=pd.DataFrame(KM.transform(X))
res=pd.concat((res,pd.DataFrame(KM.fit_predict(X))),axis=1)
res.columns=list(range(n))+["cluster"]
res.loc[:,"score"]=res.apply(lambda x: x[int(x["cluster"])],axis=1)

In [ ]:
citi_day_by_station['cluster'] = res.cluster
citi_day_by_station['km_score'] = res.score
citi_day_by_station['outlier'] = 0
citi_day_by_station.sort_values("km_score",ascending=False, inplace=True)

In [79]:
citi_day_by_station.head(3)

,bike_date,tripduration,start_station id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bikeid,birth_year,gender,start_hour,end_hour,bike_trip_count,weekday


In [ ]:
citi_day_by_station['outlier'][:10] = 1 

In [ ]:
def set_color(Data1):
    if (Data1.outlier == 1):
        return "red"
    elif (Data1.cluster == 0):
        return "blue"
    elif (Data1.cluster == 1):
        return "cyan"
    else:
        return "green"

citi_day_by_station = citi_day_by_station.assign(color=citi_day_by_station.apply(set_color, axis=1))

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = Axes3D(fig)

ax.scatter(X.taxi_trip_count, X.total_amount, X.weekend, c=citi_day_by_station.color)
ax.set_xlabel('Taxi Trip Count')
ax.set_ylabel('average total amount $')
ax.set_zlabel('weekend y / n')
# set x ticks and labels
ax.set_xticks(range(-5, 5, 1))
# change fontsize
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(10)
plt.show()

## Isolation Anomaly Detection

### Isolation Forest Anomaly Detection

In [ ]:
train,test=train_test_split(X, random_state=999, test_size=0.3)
# fit the model
clf = IsolationForest(max_samples=100, random_state=999)
clf.fit(train)
y_pred = clf.predict(X)

In [ ]:
citi_day_by_station['decision_function'] = clf.decision_function(X)

In [ ]:
citi_day_by_station.sort_values(by='decision_function').head(10)

## Look at specific areas and see what times / dates around there are anomalies
### Madison Square Gardens (40.750556, -73.993611)

In [ ]:
latlon_list = [40.750556, -73.993611]
#only keep trips within a reasonable pickup lat / lon (without converting geometries)
citi_df_total_subset = citi_day_by_station[(citi_day_by_station.start_station_latitude > (latlon_list[0] - 0.005)) & (citi_day_by_station.start_station_latitude < (latlon_list[0] + 0.005))]
citi_df_total_subset = citi_df_total_subset[(citi_df_total_subset.start_station_longitude > (latlon_list[1] - 0.005)) & (citi_df_total_subset.start_station_longitude < (latlon_list[1] + 0.005))]

In [ ]:
# convert to geopandas df
geometry = [Point(xy) for xy in zip(citi_df_total_subset.start_station_longitude, citi_df_total_subset.start_station_latitude)]

In [ ]:
crs = {'init': 'epsg:4326'}
citi_df_total_subset = GeoDataFrame(citi_df_total_subset, crs=crs, geometry=geometry)

In [ ]:
# create MSG buffer 
d = {'latitude': latlon_list[0], 'longitude': latlon_list[1]}
MSG = pd.DataFrame(data=d, index=[0])
geometry_MSG = [Point(xy) for xy in zip(MSG.longitude, MSG.latitude)]
crs = {'init': 'epsg:4326'}
MSG_gp = GeoDataFrame(crs=crs, geometry=geometry_MSG)

In [ ]:
#create buffer
MSG_gp['geometry'] = MSG_gp.geometry.buffer(.005)
MSG_gp.head()

In [ ]:
# only keep points within MSG buffer
citi_df_total_subset = citi_df_total_subset[citi_df_total_subset.geometry.intersects(MSG_gp.geometry[0])]

In [ ]:
citi_df_total_subset['hour'] = citi_df_total_subset.tpep_pickup_datetime.apply(lambda x: x.hour)

In [ ]:
#subset to 3pm - midnight hours (to only capture relevant hours)
#tlc_df_total_gp_subset = tlc_df_total_gp_subset[(tlc_df_total_gp_subset.hour >= 15) & (tlc_df_total_gp_subset.hour <= 23)]

In [ ]:
#take the mean of the taxi stats each day / hour
citi_df_MSG_mean = citi_df_total_subset.groupby(['taxi_date'], as_index=False).mean()
citi_df_MSG_mean.head()

In [ ]:
citi_df_MSG_mean['day_of_week'] = citi_df_MSG_mean.taxi_date.apply(lambda x: x.weekday())

In [ ]:
#control for Friday / Saturday nights
citi_df_MSG_mean['weekend'] = 0
citi_df_MSG_mean['weekend'][(citi_df_MSG_mean['day_of_week'] == 4) | (citi_df_MSG_mean['day_of_week'] == 5)] = 1

In [ ]:
#add holiday dummy variable
citi_df_MSG_mean['holiday'] = 0
citi_df_MSG_mean['holiday'][citi_df_MSG_mean['taxi_date'].isin(holidays_all.index.date)] = 1

In [ ]:
# read in NYC street shapefile
'''
shapefile taken from https://geo.nyu.edu/catalog/nyu_2451_34565

'''
ny_streets_raw = gp.read_file(proj_folder + '/data/nyu_2451_34565/nyu_2451_34565.shp')
ny_streets_raw = ny_streets_raw.to_crs(epsg=4326)
# only keep points within MSG buffer
ny_streets = ny_streets_raw[ny_streets_raw.geometry.intersects(MSG_gp.geometry[0])]

In [ ]:
# is there a way to easily plot each color differently? I have this color column highlighting outliers!

MSG_sample = citi_df_MSG_mean.sample(n=1000)
#MSG_sample.lon = MSG_sample.geometry.x
#MSG_sample.lat = MSG_sample.geometry.y
f, (ax1) = plt.subplots(figsize=(12,8))
MSG_sample.plot(alpha=0.5, ax=ax1, color = 'red')
ny_streets['geometry'].plot(alpha=1, linewidth=0.25, ax=ax1, color = '0')
ax1.set_title("Plot of Taxi Pickups, Around MSG", fontsize=20)
plt.xlabel("longitude")
plt.ylabel("latitude")

## MSG anomaly analysis (isolation forest)

In [ ]:
X = tlc_df_total_gp_subset[cluster_features]
X.head(2)

In [ ]:
#normalize columns
for i in X.columns:
    X[i] = (X[i] - X[i].mean()) / X[i].std()

In [ ]:
train,test=train_test_split(X, random_state=999, test_size=0.3)
# fit the model
clf = IsolationForest(max_samples=100, random_state=999)
clf.fit(train)
y_pred = clf.predict(X)

In [ ]:
citi_df_total_subset['decision_function'] = clf.decision_function(X)

In [ ]:
citi_df_total_subset.sort_values(by='decision_function').head(10)

### K-Means Cluster Model Anomaly Detection

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for n_clusters in range_n_clusters:
    km = KMeans(n_clusters=n_clusters, random_state=324)
    cluster_labels = km.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters ={},".format(n_clusters)+" the average silhouette_score is :{}".format(silhouette_avg))

In [ ]:
n=3
KM=KMeans(n_clusters=n,random_state=999)
scor=KM.fit_predict(X)

res=pd.DataFrame(KM.transform(X))
res=pd.concat((res,pd.DataFrame(KM.fit_predict(X))),axis=1)
res.columns=list(range(n))+["cluster"]
res.loc[:,"score"]=res.apply(lambda x: x[int(x["cluster"])],axis=1)

In [ ]:
citi_df_total_subset['cluster'] = res.cluster
citi_df_total_subset['km_score'] = res.score
citi_df_total_subset['outlier'] = 0
citi_df_total_subset.sort_values("km_score",ascending=False, inplace=True)

In [ ]:
citi_df_total_subset['outlier'][:10] = 1 

In [ ]:
def set_color(Data1):
    if (Data1.outlier == 1):
        return "red"
    elif (Data1.cluster == 0):
        return "blue"
    elif (Data1.cluster == 1):
        return "cyan"
    else:
        return "green"

citi_df_total_subset = citi_df_total_subset.assign(color=citi_df_total_subset.apply(set_color, axis=1))

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = Axes3D(fig)

ax.scatter(X.taxi_trip_count, X.total_amount, X.weekend, c=citi_df_total_subset.color)
ax.set_xlabel('Taxi Trip Count')
ax.set_ylabel('average total amount $')
ax.set_zlabel('weekend y / n')
# set x ticks and labels
ax.set_xticks(range(-5, 5, 1))
# change fontsize
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(10)
plt.show()